In [0]:
import requests
import re
import uuid
from bs4 import BeautifulSoup
import os

In [0]:
if not os.path.exists(images_dir):
    os.makedirs(images_dir)


In [0]:
import sys
import os
import subprocess

ここまで実行したらimages_dir下に新しいサブディレクトリを作って画像をそれぞれいれる

In [0]:
backup_dir = './model'

In [0]:
def cmd(cmd):
   p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
   p.wait()
   stdout, stderr = p.communicate()
   return stdout.rstrip()

dirs = cmd("ls "+"./image_data")
labels = dirs.splitlines()

if os.path.exists(backup_dir):
   cmd("rm  -rf "+backup_dir)

# make directories
os.makedirs(backup_dir)
labelsTxt_backup = open(backup_dir + '/labels.txt','w')
classNo=0
for label in labels:
   labelsTxt_backup.write(label.decode('utf-8')+"\n")
   classNo += 1

NUM_CLASSES = classNo
print("class number=" + str(NUM_CLASSES))
labelsTxt_backup.close()

In [0]:
!cat ./model/labels.txt

In [0]:
import tensorflow as tf
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.python.keras import optimizers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time

In [0]:
def list_pictures(directory, ext=('jpg', 'jpeg', 'bmp', 'png', 'ppm', 'tif',
                                  'tiff')):
    """Lists all pictures in a directory, including all subdirectories.
    # Arguments
        directory: string, absolute path to the directory
        ext: tuple of strings or single string, extensions of the pictures
    # Returns
        a list of paths
    """
    ext = tuple('.%s' % e for e in ((ext,) if isinstance(ext, str) else ext))
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if f.lower().endswith(ext)]

In [0]:
labels = []
with open(backup_dir + '/labels.txt','r') as f:
   for line in f:
       labels.append(line.rstrip())
print(labels)

In [0]:
X = []
Y = []

label_num = 0
for label in labels:
   print(label)
   for picture in list_pictures('./image_data/' + label):
       img = img_to_array(load_img(picture, target_size=(64,64)))
       X.append(img)
       Y.append(label_num)
       
   label_num += 1

In [0]:
# arrayに変換
X = np.asarray(X)
Y = np.asarray(Y)

# 画素値を0から1の範囲に変換
X = X.astype('float32')
X = X / 255.0

# クラスの形式を変換
Y = np_utils.to_categorical(Y, NUM_CLASSES)

# 学習用データとテストデータ
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=111)

In [0]:
# 学習率
LEARNING_RATE = 0.001
# バッチサイズ
BATCH_SIZE = 5
# エポック（世代数）
EPOCHS = 100

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', 
                input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

sgd = optimizers.SGD(lr=LEARNING_RATE)

model.compile(loss='categorical_crossentropy',
             optimizer='SGD',
             metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
# モデルの保存
model_json_str = model.to_json()
with open(backup_dir + '/mnist_deep_model.json', 'w') as f:
   f.write(model_json_str)

# 重みデータのバックアップ
cb_cp = tf.keras.callbacks.ModelCheckpoint(backup_dir + '/weights.{epoch:02d}.hdf5', verbose=1, save_weights_only=True)
# TensorBoard用のデータ
cb_tf = tf.keras.callbacks.TensorBoard(log_dir=backup_dir + '/tensorBoard', histogram_freq=0)

start = time.time()
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                  validation_data = (X_test, y_test), verbose = 1,  callbacks=[cb_cp, cb_tf])
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['acc', 'val_acc'], loc='lower right')
plt.show()

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()